In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error as mse

from scipy import sparse
from itertools import product

In [14]:
transactions = pd.read_csv('../transactions.csv')
customers_gender = pd.read_csv('../customers_gender_train.csv')
sex_best = pd.read_csv("../task1/compose-xgb0.7-rf0.2-logreg.csv")

In [15]:
#sex_all = pd.concat([customers_gender, sex_best])

In [16]:
cuses_test = list(set(transactions.customer_id.unique().tolist()).difference(customers_gender.customer_id.unique()))
all_cuses = transactions.customer_id.unique()
all_mcc = transactions.mcc_code.unique()

In [17]:
transactions = transactions[transactions.amount < 0].copy()
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

In [18]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,day
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,0
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN,1
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN,1
4,39026145,2 15:33:42,5499,1010,-920.83,NaN,2
5,39026145,2 15:53:49,5541,1010,-14643.37,NaN,2


In [19]:
#transactions = pd.merge(transactions, sex_all)

In [20]:
transactions.day += 29 - transactions['day'].max()%30

In [21]:
transactions['month_num'] = (transactions.day) // 30
#transactions['year_num'] = (transactions.day) // 365

transactions['year_num'] = (transactions.month_num) // 12

In [22]:
test_transactions = transactions[transactions.month_num == 15]
train_transactions = transactions[transactions.month_num < 15]

In [23]:
train_transactions.tail()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,day,month_num,year_num
6849254,61870738,420 16:07:56,5311,1110,-4222.32,549010,443,14,1
6849255,61870738,420 16:12:11,5999,1010,-4491.83,10288232,443,14,1
6849257,61870738,420 19:06:60,4829,2370,-89207.77,888905,443,14,1
6849258,61870738,421 10:05:49,5499,1010,-7231.85,10217113,444,14,1
6849259,61870738,422 10:22:34,6011,2010,-67377.47,582861,445,14,1


In [24]:
test_transactions = test_transactions.set_index('customer_id')
test_transactions = test_transactions.loc[cuses_test]
test_transactions = test_transactions.reset_index()

In [25]:
grid = list(product(*[all_cuses, all_mcc, range(7, 15)]))
train_grid = pd.DataFrame(grid, columns = ['customer_id', 'mcc_code', 'month_num'])

In [26]:
test_grid = list(product(*[cuses_test, all_mcc]))       
test_grid = pd.DataFrame(test_grid, columns = ['customer_id', 'mcc_code'])
test_grid['month_num'] = 15

In [27]:
test_grid.head()

,customer_id,mcc_code,month_num
0,55705601,4814,15
1,55705601,6011,15
2,55705601,4829,15
3,55705601,5499,15
4,55705601,5541,15


In [28]:
%%time
test = pd.merge(test_grid,
         test_transactions.groupby(['year_num', 'month_num', 'customer_id', 'mcc_code'])[['amount']].sum().reset_index(),
         how='left').fillna(0)

CPU times: user 307 ms, sys: 0 ns, total: 307 ms
Wall time: 305 ms


In [29]:
test['year_num'] = (test.month_num) // 12
test.head(5)

,customer_id,mcc_code,month_num,year_num,amount
0,55705601,4814,15,1,-26951.00
1,55705601,6011,15,1,-190902.84
2,55705601,4829,15,1,-113418.75
3,55705601,5499,15,1,-1347.55
4,55705601,5541,15,1,0.00


In [17]:
%%time
train = pd.merge(train_grid,
         train_transactions.groupby(['year_num', 'month_num', 'customer_id', 'mcc_code'])[['amount']].sum().reset_index(),
         how='left').fillna(0)

CPU times: user 10.3 s, sys: 1.93 s, total: 12.2 s
Wall time: 12.2 s


In [18]:
train['year_num'] = (train.month_num) // 12

In [19]:
%%time
for month_shift in range(1, 5):
    train_shift = train.copy()
    train_shift['month_num'] = train_shift['month_num'] + month_shift
    train_shift['year_num'] = (train_shift.month_num) // 12
    train_shift = train_shift.rename(columns={"amount" : 'amount_{0}'.format(month_shift)})  
    train_shift = train_shift[['year_num', 'month_num', 'customer_id', 'mcc_code', 'amount_{0}'.format(month_shift)]]

    train = pd.merge(train, train_shift, 
                                  on=['year_num', 'month_num', 'customer_id', 'mcc_code'], how='left').fillna(0)
    test = pd.merge(test, train_shift, 
                                 on=['year_num', 'month_num', 'customer_id', 'mcc_code'], how='left').fillna(0)

CPU times: user 1min 28s, sys: 19.3 s, total: 1min 47s
Wall time: 1min 47s


In [20]:
trainn = train[train.month_num < 14]
val = train[train.month_num == 14]

In [21]:
train_all = train[train.month_num < 15]

In [52]:
test.head(10)

,customer_id,mcc_code,month_num,year_num,amount,amount_1,amount_2,amount_3,amount_4
0,55705601,4814,15,1,-26951.00,-13475.50,-41549.45,-112981.92,-37282.21
1,55705601,6011,15,1,-190902.84,-132509.02,-496347.38,-384051.60,-312182.29
2,55705601,4829,15,1,-113418.75,-3368.87,-206646.71,0.00,-73558.23
3,55705601,5499,15,1,-1347.55,-2627.72,0.00,-2672.64,-2829.85
4,55705601,5541,15,1,0.00,0.00,0.00,0.00,0.00
5,55705601,5200,15,1,0.00,0.00,0.00,0.00,0.00
6,55705601,5732,15,1,0.00,0.00,0.00,0.00,0.00
7,55705601,6010,15,1,0.00,0.00,0.00,0.00,0.00
8,55705601,5331,15,1,0.00,0.00,0.00,0.00,0.00
9,55705601,5912,15,1,0.00,-3718.56,0.00,-19067.82,0.00


index_selecting

In [46]:
trainn_idx = trainn[(trainn.amount_1 != 0) & (trainn.amount_2 != 0) & (trainn.amount_3 != 0) & (trainn.amount_4 != 0)].index

In [50]:
trainn2 = trainn.loc[trainn_idx]

,customer_id,mcc_code,month_num,year_num,amount,amount_1,amount_2,amount_3,amount_4
4,39026145,4814,11,0,-21336.23,-56237.76,-7860.72,-20662.44,-20213.26
5,39026145,4814,12,1,-32565.80,-21336.23,-56237.76,-7860.72,-20662.44
6,39026145,4814,13,1,-32565.81,-32565.80,-21336.23,-56237.76,-7860.72
12,39026145,6011,11,0,-145984.52,-307690.46,-604151.34,-1729355.14,-723184.88
13,39026145,6011,12,1,-653561.50,-145984.52,-307690.46,-604151.34,-1729355.14


In [114]:
########

null_index_tr = trainn[trainn.amount_1 == 0][trainn.amount_2 == 0][trainn.amount_3 == 0][trainn.amount_4 == 0].index

#train = train.drop(null_index)
#train = train.reset_index().drop(["index"], axis=1)

/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [23]:
hasher = FeatureHasher(n_features=10000, input_type='string')

In [25]:
%%time
train_all_sparse = \
    hasher.fit_transform(train_all[['year_num', 'month_num', 'customer_id', 'mcc_code']].astype(str).as_matrix())

CPU times: user 5min, sys: 2.99 s, total: 5min 3s
Wall time: 5min 2s


In [26]:
%%time
train_sparse = \
    hasher.transform(trainn[['year_num', 'month_num', 'customer_id', 'mcc_code']].astype(str).as_matrix())

CPU times: user 4min 26s, sys: 1.74 s, total: 4min 28s
Wall time: 4min 27s


In [29]:
%%time
val_sparse = \
    hasher.transform(val[['year_num', 'month_num', 'customer_id', 'mcc_code']].astype(str).as_matrix())

CPU times: user 38 s, sys: 58 ms, total: 38.1 s
Wall time: 38 s


In [30]:
%%time
test_sparse = \
    hasher.transform(test[['year_num', 'month_num', 'customer_id', 'mcc_code']].astype(str).as_matrix())

CPU times: user 7.68 s, sys: 9 ms, total: 7.69 s
Wall time: 7.65 s


In [31]:
%time
train_sparse1 = sparse.hstack([train_sparse,
                              np.log(np.abs(trainn[['amount_1', 'amount_2', 'amount_3', 'amount_4']]) + 1).as_matrix()
                             ])

val_sparse1 = sparse.hstack([val_sparse,
                             np.log(np.abs(val[['amount_1', 'amount_2', 'amount_3', 'amount_4']]) + 1).as_matrix()
                            ])

test_sparse1 = sparse.hstack([test_sparse,
                             np.log(np.abs(test[['amount_1', 'amount_2', 'amount_3', 'amount_4']]) + 1).as_matrix()
                            ])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 20 µs


In [32]:
train_all_sparse1 = sparse.hstack([train_all_sparse,
                                  np.log(np.abs(train_all[['amount_1', 'amount_2', 'amount_3', 'amount_4']]) + 1).as_matrix()
                             ])

In [33]:
%%time
shift = 1
clf = LinearRegression(n_jobs=22)
clf.fit(train_sparse1, np.log(-trainn['amount'] + shift))

CPU times: user 45.8 s, sys: 2.91 s, total: 48.7 s
Wall time: 48.7 s


In [34]:
pred = np.e ** clf.predict(val_sparse1) - shift
val["pred_amount"] = pred

/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [39]:
val['pred_amount'].loc[val["pred_amount"] < 0] = 0

/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [40]:
val[val.pred_amount < 0].shape

(0, 10)

In [41]:
true_log_amount = np.log(-val['amount'] + shift)
pred_log_amount = np.log(val['pred_amount'] + shift)

In [42]:
print np.sqrt(mse(true_log_amount,pred_log_amount))

1.32834258961


In [ ]:
1.35088736297

### Train on all set

In [42]:
%%time
shift = 1
clf = LinearRegression(n_jobs=22)
clf.fit(train_all_sparse1, np.log(-train_all['amount'] + shift))

CPU times: user 10.4 s, sys: 4.42 s, total: 14.8 s
Wall time: 14.8 s


In [43]:
%%time
test['volume'] = np.e ** clf.predict(test_sparse1) - shift
#test[['customer_id', 'mcc_code', 'volume']].to_csv('baseline_c.csv', index=False)

CPU times: user 61 ms, sys: 0 ns, total: 61 ms
Wall time: 59.7 ms


In [44]:
test.head()

,customer_id,mcc_code,month_num,year_num,amount,amount_1,amount_2,amount_3,amount_4,volume
0,55705601,4814,15,1,-26951.00,-13475.50,-41549.45,-112981.92,-37282.21,1.289558e+05
1,55705601,6011,15,1,-190902.84,-132509.02,-496347.38,-384051.60,-312182.29,2.825573e+06
2,55705601,4829,15,1,-113418.75,-3368.87,-206646.71,0.00,-73558.23,1.856789e+04
3,55705601,5499,15,1,-1347.55,-2627.72,0.00,-2672.64,-2829.85,8.782244e+02
4,55705601,5541,15,1,0.00,0.00,0.00,0.00,0.00,3.112507e+00


In [45]:
test[test.volume < 0].head()

,customer_id,mcc_code,month_num,year_num,amount,amount_1,amount_2,amount_3,amount_4,volume


In [46]:
#test['volume'].loc[test["volume"] < 0] = 0
test[['customer_id', 'mcc_code', 'volume']].to_csv('baseline_correct_range5_month_mcc_only.csv', index=False)

### xgb

In [92]:
import xgboost as xgb

In [152]:
param = {}
param['max_depth'] = 3
param['booster'] = 'gbtree'
param['objective'] = 'reg:linear'
param['eval_metric'] = 'rmse'
param['eta'] = 0.05
param['subsample'] = 0.7
param['colsample_bytree'] = 0.8
param['colsample_bylevel'] = 0.8

numround = 100

In [96]:
train.amount_1 = train.amount_1.apply(lambda x: np.log(1-x))

In [97]:
train.amount_2 = train.amount_2.apply(lambda x: np.log(1-x))
train.amount_3 = train.amount_3.apply(lambda x: np.log(1-x))

In [98]:
test.amount_1 = test.amount_1.apply(lambda x: np.log(1-x))
test.amount_2 = test.amount_2.apply(lambda x: np.log(1-x))
test.amount_3 = test.amount_3.apply(lambda x: np.log(1-x))

In [132]:
X_train = train[["mcc_code", "month_num", "year_num", "amount_1", "amount_2", "amount_3"]]
X_test = test[["mcc_code", "month_num", "year_num", "amount_1", "amount_2", "amount_3"]]

In [136]:
Xdatatrain = xgb.DMatrix(data=X_train, label = np.log(-train['amount'] + 1))
Xdatatest = xgb.DMatrix(data=X_test)

plst = list(param.items())

In [151]:
%%time
xgb.cv(param, Xdatatrain, num_boost_round=1000, nfold=3, metrics="rmse", \
       verbose_eval=1, as_pandas=False, seed=27)

[0]	train-rmse:1.94011+0.000253142	test-rmse:1.94011+0.000483145
[1]	train-rmse:1.89227+0.0119002	test-rmse:1.89228+0.0113567
[2]	train-rmse:1.86802+0.0123896	test-rmse:1.86806+0.0118619
[3]	train-rmse:1.83612+0.0288408	test-rmse:1.83615+0.0283525
[4]	train-rmse:1.78319+0.0267665	test-rmse:1.78325+0.0262959
[5]	train-rmse:1.73402+0.0250776	test-rmse:1.73407+0.0246411
[6]	train-rmse:1.69534+0.0137811	test-rmse:1.69537+0.0134292
[7]	train-rmse:1.66524+0.0209556	test-rmse:1.66528+0.020499
[8]	train-rmse:1.62654+0.0211091	test-rmse:1.62657+0.0206392
[9]	train-rmse:1.59659+0.0271763	test-rmse:1.5966+0.0267039
[10]	train-rmse:1.57076+0.0314217	test-rmse:1.57076+0.0309225
[11]	train-rmse:1.54287+0.0297486	test-rmse:1.54292+0.0292376
[12]	train-rmse:1.51161+0.027394	test-rmse:1.51166+0.0268867
[13]	train-rmse:1.49562+0.0295131	test-rmse:1.49566+0.029038
[14]	train-rmse:1.47233+0.0220193	test-rmse:1.47237+0.0215726
[15]	train-rmse:1.45313+0.0174394	test-rmse:1.45313+0.0170374
[16]	train-rmse:1.

KeyboardInterrupt: 

In [153]:
%%time
bst = xgb.train(plst, Xdatatrain, numround, verbose_eval = 10)

CPU times: user 2h 57s, sys: 6.94 s, total: 2h 1min 4s
Wall time: 5min 42s


In [154]:
y_pred_xgb = bst.predict(Xdatatest)

In [155]:
min(y_pred_xgb), max(y_pred_xgb)

(-0.10811269, 16.312561)

In [156]:
res_xgb = np.e ** bst.predict(Xdatatest) - shift

In [157]:
res_xgb

array([  1.05005371e+03,   7.02642871e+03,   5.57351641e+04, ...,
        -2.17597485e-02,   1.59621239e-03,   1.00523233e-02], dtype=float32)

In [158]:
test['volume'] = res_xgb

In [159]:
test['volume'].loc[test["volume"] < 0] = 0

In [149]:
test[['customer_id', 'mcc_code', 'volume']].to_csv('baseline_xgb.csv', index=False)

## compose

In [162]:
y_compose = 0.7*(np.e ** clf.predict(test_sparse) - shift) + 0.3*(np.e ** bst.predict(Xdatatest) - shift)

In [163]:
test['volume'] = y_compose

In [164]:
test['volume'].loc[test["volume"] < 0] = 0

In [166]:
test[['customer_id', 'mcc_code', 'volume']].to_csv('baseline_xgb_linreg07.csv', index=False)

## rf

In [160]:
from sklearn.ensemble import RandomForestClassifier

In [161]:
X_train = train[["mcc_code", "month_num", "year_num", "amount_1", "amount_2", "amount_3"]]

In [128]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=2, n_jobs=-1, oob_score=True)
rf.fit(X_train, list(np.log(-train['amount'] + 1)))

ValueError: Unknown label type: array([[ 10.93736148],
       [  9.96820871],
       [  8.4102403 ],
       ..., 
       [  0.        ],
       [  0.        ],
       [  0.        ]])